In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch
from tqdm.auto import tqdm
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch import optim
plt.rcParams['font.family'] = 'Malgun Gothic'
# 데이터 불러오기
df_all=pd.read_csv('../data_v3/감성점수와 일별뉴스 합친거.csv',encoding='utf-8')
df = df_all.iloc[:,[1,2,5]]

In [3]:
# 데이터 불러오기
df_all=pd.read_csv('../data_v3/감성점수와 일별뉴스 합친거.csv',encoding='utf-8')
df = df_all.iloc[:,[1,2,5]]

df_all=pd.read_excel('../data_v3/전국경락단가.xlsx')

df_all

df['경락단가_평균']=df_all['price']

df_all

df.columns=['경락단가', '도매가격', 'day_sentiment']

df

df

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  """


,경락단가,도매가격,day_sentiment
0,3750.366093,4104,0.000000
1,3547.574932,4049,0.000000
2,3696.060897,3920,-0.104167
3,3770.156364,3917,-0.104167
4,3809.974684,4090,-0.104167
...,...,...,...
620,4547.071429,4718,0.175000
621,4373.212544,4749,-0.116270
622,4278.649485,4820,-0.116270
623,4322.080986,4919,-0.116270


In [4]:
# 데이터 불러오기
df_all=pd.read_csv('../data_v3/감성점수와 일별뉴스 합친거.csv',encoding='utf-8')
df = df_all.iloc[:,[1,2,5]]
df_all=pd.read_excel('../data_v3/전국경락단가.xlsx')
df['경락단가_평균']=df_all['price']
df.columns=['경락단가', '도매가격', 'day_sentiment']

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  after removing the cwd from sys.path.


In [5]:
df_data=df[['도매가격','day_sentiment','도매가격']]
title='도매가격_감성점수'
term_np=np.array([])
MAE_np=np.array([])
MSE_np=np.array([])
RMSE_np=np.array([])
MAPE_np=np.array([])

In [6]:
for i in range(2,22):
    for j in range(2,8):
        # 5일간의 데이터가 입력으로 들어가고 batch size는 임의로 지정
        CFG={'TRAIN_WINDOW_SIZE':i,
           'PREDICT_SIZE':j ,
            'how_inputdim':2,
            'hidden_dim':10,
            'LEARNING_RATE':0.001,
            'nb_epochs':100,
            'BATCH_SIZE':25}
        term="{}train_size-{}predict_size".format(CFG['TRAIN_WINDOW_SIZE'],CFG['PREDICT_SIZE'])
        print(term)
    
        # 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
        train_size = int(len(df)*0.7)
        train_set = df_data[0:train_size]  
        test_set = df_data[train_size-CFG['TRAIN_WINDOW_SIZE']:]

        # Input scale
        scaler_x = MinMaxScaler()
        scaler_x.fit(train_set.iloc[:, :-1])

        train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
        test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])

        # Output scale
        scaler_y = MinMaxScaler()
        scaler_y.fit(train_set.iloc[:, [-1]])

        train_set.iloc[:, -1:] = scaler_y.transform(train_set.iloc[:, -1:])
        test_set.iloc[:, -1:] = scaler_y.transform(test_set.iloc[:, -1:])


        # 데이터셋 생성 함수
        def build_dataset(time_series,train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
            dataX = []
            dataY = []
            window_size = train_size + predict_size
            for i in range(len(time_series) - window_size + 1):
                _x = time_series[i:i+train_size, :-1]
                _y = time_series[i+train_size:i+train_size+predict_size, [-1]]
                # print(_x, "-->",_y)
                dataX.append(_x)
                dataY.append(_y)

            return np.array(dataX), np.array(dataY)

        trainX, trainY = build_dataset(np.array(train_set))
        testX, testY = build_dataset(np.array(test_set))

        class CustomDataset(Dataset):
            def __init__(self, X, Y):
                self.X = X
                self.Y = Y

            def __getitem__(self, index):
                if self.Y is not None:
                    return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
                return torch.Tensor(self.X[index])

            def __len__(self):
                return len(self.X)

        train_dataset = CustomDataset(trainX, trainY)
        train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

        val_dataset = CustomDataset(testX, testY)
        val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

        # test_dataset = CustomDataset(test_input, None)
        # test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

        # 모델선언

        class BaseModel(nn.Module):
            def __init__(self, input_size=CFG['how_inputdim'], hidden_size=CFG['hidden_dim'], output_size=CFG['PREDICT_SIZE']):
                super(BaseModel, self).__init__()
                self.num_classes = num_classes
                self.num_layers = num_layers
                self.input_size = input_size
                self.hidden_size = hidden_size
                self.seq_length = seq_length

                self.gru = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                                 num_layers=num_layers,batch_first=True)
                self.fc_1 = nn.Linear(hidden_size, 128)
                self.fc = nn.Linear(128, num_classes)
                self.relu = nn.ReLU()

            def forward(self, x):
                h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
                output, (hn) = self.gru(x, (h_0))
                hn = hn.view(-1, self.hidden_size)
                out = self.relu(hn)
                out = self.fc_1(out)
                out = self.relu(out)
                out = self.fc(out)
                return out

            def init_hidden(self, batch_size, device):
                # Initialize hidden state and cell state
                return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                        torch.zeros(1, batch_size, self.hidden_size, device=device))

        # 모델학습
        
        def train(model, optimizer, train_loader, val_loader, device):
            model.to(device)
            criterion = nn.MSELoss().to(device)
            best_loss = 9999999
            best_model = None

            for epoch in range(1, CFG['nb_epochs']+1):
                model.train()
                train_loss = []
                train_mae = []
                for X, Y in tqdm(iter(train_loader)):
                    X = X.to(device)
                    Y = Y.to(device)
                    optimizer.zero_grad()

                    output = model(X)
                    loss = criterion(output, Y.squeeze())

                    loss.backward()
                    optimizer.step()

                    train_loss.append(loss.item())

                val_loss = validation(model, val_loader, criterion, device)
                print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

                if best_loss > val_loss:
                    best_loss = val_loss
                    best_model = model
                    print('Model Saved')
            return best_model

        def validation(model, val_loader, criterion, device):
            model.eval()
            val_loss = []

            with torch.no_grad():
                for X, Y in tqdm(iter(val_loader)):
                    X = X.to(device)
                    Y = Y.to(device)

                    output = model(X)
                    loss = criterion(output, Y.squeeze())

                    val_loss.append(loss.item())
            return np.mean(val_loss)

        model = BaseModel()
        optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
        infer_model = train(model, optimizer, train_loader, val_loader, device)

        def inference(model, test_loader, device):
            model.eval()
            predictions=[]
            with torch.no_grad():
                for X, Y in tqdm(iter(test_loader)):
                    X = X.to(device)
                    Y = Y.to(device)

                    output = model(X)
                    # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
                    output = output.cpu().numpy()
                    predictions.extend(output)

            return np.array(predictions)

        answer_np = np.empty((0,CFG['PREDICT_SIZE']))
        
        for val in val_loader:
            answer_np = np.vstack((answer_np, val[1].view(-1,CFG['PREDICT_SIZE'])))

        pred = inference(infer_model,val_loader , device)
        print(pred.shape)

        pred=scaler_y.inverse_transform(pred)
     
        # 눤래 정답을 inverse scaling

        answer_np=scaler_y.inverse_transform(answer_np)    
        # 결과 후처리
        answer_np = np.round(answer_np, 0).astype(int)
        print("answer_np", answer_np)

        # 결과 출력
        #MAE
        def MAE(true, pred):
            return np.mean(np.abs(true-pred))

        print('MAE SCORE : ', MAE(answer_np, pred))
        MAE_SCORE=MAE(answer_np, pred)

        #MSE
        from sklearn.metrics import mean_squared_error

        print("MSE",mean_squared_error(answer_np,pred))
        MSE=mean_squared_error(answer_np,pred)
        #RMSE
        import numpy as np
        from sklearn.metrics import mean_squared_error

        MSE = mean_squared_error(answer_np,pred)
        print("RMSE",np.sqrt(MSE))
        # sklearn 은 mse만 제공하기 때문에 rmse는 직접 만들어 써야한다.
        RMSE=np.sqrt(MSE)
        #MAPE
        import numpy as np

        def MAPE(y_test, y_pred):
            return np.mean(np.abs((answer_np - pred) / answer_np)) * 100

        print("MAPE",MAPE(answer_np, pred))
        MAPE_SCORE=MAPE(answer_np, pred)

        #결과값 데이터프레임으로 저장
        term_np=np.append(term_np,term)
        MAE_np=np.append(MAE_np,MAE_SCORE )
        MSE_np=np.append(MSE_np,MSE )
        RMSE_np=np.append(RMSE_np,RMSE )
        MAPE_np=np.append(MAPE_np,MAPE_SCORE )

        # 그래프 그리기
        answer=np.mean(answer_np, axis=1)
        prediction = np.mean(pred, axis=1)

        fig = plt.figure(figsize=(8,3))
        plt.plot(np.arange(len(answer)), answer, label = 'true')
        plt.plot(np.arange(len(prediction)), prediction, label = 'pred')
        plt.legend()
        plt.title(title)
        plt.savefig("../결과/GRU_감성점수_graph/{} {}train_size {}predict_size.png".format(title,CFG['TRAIN_WINDOW_SIZE'],CFG['PREDICT_SIZE']), dpi=300)
        

2train_size-2predict_size


C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on 

NameError: name 'num_classes' is not defined

### 결과 저장

In [ ]:
df_result=pd.DataFrame()

In [ ]:
df_result['term']=term_np
df_result['MAE']=MAE_np
df_result['MSE']=MSE_np
df_result['RMSE']=RMSE_np
df_result['MAPE']=MAPE_np

In [ ]:
df_result.to_csv("../결과/GRU_감성점수_result/result.csv".format(title,CFG['TRAIN_WINDOW_SIZE'],CFG['PREDICT_SIZE']),encoding="utf-8")

In [ ]:
answer_np.shape

In [ ]:
pred.shape

In [ ]:
 for val in val_loader:
    print(val[0])


In [ ]:
answer